# Market Making Engine

here's the framework

# TODO

* add speed protection : don't add orders if moves are too fast
* add general wallet
* be less agressive with the exception catch : let the program die when there is a real syntax error !
* Store datas ( like buy / sell / wallet evolution ) ( so we can track/monitor progress )
* explore different limit / spread
* Set the pair in the engine rather than in the market maker ? ( it's at two different places now for backtest )
* factorize engines
* what if an order is canceled outside the bot ?
* add setup var like quote amount, fees, etc...
* assert if limit < 2
* don't use a global var to trigger logs ( sorry )
* sometimes, the buy price is far from the sell price :(
* Quote management ( don't buy if available quote quant < buy amount )
* Is there a floating point rounding bug on the amount ?
   => it looks like the exchange and the engine don't have exactly the same count :(
* get rich... ( still very wip )...

# Done :

* Paper trading : add wallet display
* add backtest engine
* remove while parsing sell list bug
* Cancel orders
* what if there is only buy orders ?
* try live DONE :)
* fees
* Deal with full sell / buy lists
* Make a special mode for backtest without logs, and that return the gain, to find the best parameters
* limit float display
* Backtest : save candles as files, so we don't redownload data everytime...
* extract credentials to another file


In [126]:
import ccxt
import time
from datetime import datetime
import traceback
import pickle

In [ ]:
%run Credentials.ipynb

In [119]:
exchange = getattr( ccxt, "binance")({
    'apiKey': API_Key,
    'secret': Secret_Key,
    'timeout': 30000,
    'enableRateLimit': True,
})

In [120]:
logOn = True

In [121]:
def log( txt, color = "black"):
    if logOn == False:
        return
    text = ""
    if ( color == "green" ):
        text = "\x1b[32m"
    if ( color == "red"):
        text = "\x1b[31m"
    if ( color == "black"):
        text = "\x1b[30m"
    if ( color == "blue"):
        text = "\x1b[34m"
    if ( color =="bgRed"):
        text = "\x1b[41m\x1b[30m"  # bg Red + Black
    text += datetime.now().strftime('%d-%m-%Y %H:%M:%S') + " - "
    text += txt
    text += "\x1b[0m" # reset all
    print(text)

In [122]:
class realTimePaperEngine:
    buyString = "buy"
    sellString = "sell"
    def __init__(self):
        self.reset()
    def reset(self):
        self.asset = 0
        self.availableAsset = 0
        self.initialQuote = 1.0
        self.quote = self.initialQuote
        self.fee = 0
        self.feePercent = 0.0750 / 100.0 # binance : 0.075 %
        self.activeBuyOrder = 0
        self.activeSellOrder = 0
        self.completedBuyOrder = 0
        self.completedSellOrder = 0
        self.gain = 0
        self.gainAfterFees = 0
    def getGain(self):
        return self.gain
    def getGainAfterFees(self):
        return self.gainAfterFees
    def getPrice(self):
        return self.close
    def getAvailableAsset(self):
        return self.availableAsset
    def update(self, pair):
        self.candle = exchange.fetch_ohlcv(pair, timeframe = '1m', limit=1)
        self.open  = self.candle[0][1]
        self.high  = self.candle[0][2]
        self.low   = self.candle[0][3]
        self.close = self.candle[0][4]
        log("current close : {:.8f} - nb active Buy : {} - nb active Sell {} - nb completed Buy : {} - nb completed Sell {}"
                .format(self.close, self.activeBuyOrder, self.activeSellOrder, self.completedBuyOrder, self.completedSellOrder))
        return True
    def buy( self, pair, amount, price):
        order = [ pair, amount, price, realTimePaperEngine.buyString ]
        self.activeBuyOrder += 1
        log ( "putting buy order for {:.8f} at price {:.8f} ".format( amount, price), "green")
        return order
    def sell( self, pair, amount, price):
        order = [ pair, amount, price, realTimePaperEngine.sellString ]
        self.activeSellOrder += 1
        self.availableAsset -= amount
        log( "putting sell order for {:.8f} at price {:.8f} ".format( amount, price), "red")
        return order
    def walletDisplay(self):
        totalInQuote = self.quote + self.asset * self.getPrice()
        self.gain = totalInQuote - self.initialQuote
        self.gainAfterFees = self.gain - self.fee;
        log("----> Wallet->Quote: {:.8f} - Asset: {:.8f} - availAsset: {:.8f} - Total: {:.8f}"
            .format(self.quote, self.asset, self.availableAsset, totalInQuote),"blue")
        log("----> Wallet->Fee: {:.8f} - gain: {:.8f} - gain after fee: {:.8f}"
            .format(self.fee, self.gain, self.gainAfterFees),"blue")
    def onBuyOrderDone(self, orderPrice, orderAmount, quoteAmount):
        self.quote -= quoteAmount
        self.asset += orderAmount
        self.availableAsset += orderAmount
        self.activeBuyOrder -= 1
        self.completedBuyOrder += 1
        self.fee += quoteAmount * self.feePercent
        log("*** buy order completed for {:.8f} at price {:.8f} ".format( orderAmount, orderPrice), "green")
        self.walletDisplay()
    def onSellOrderDone(self, orderPrice, orderAmount, quoteAmount):
        self.quote += quoteAmount
        self.asset -= orderAmount
        self.activeSellOrder -= 1
        self.completedSellOrder += 1
        self.fee += quoteAmount * self.feePercent
        log("*** sell order completed for {:.8f} at price {:.8f} ".format( orderAmount, orderPrice), "red")
        self.walletDisplay()        
    def isOrderDone( self, order):
        orderType = self.getOrderType(order)
        orderPrice = self.getOrderPrice(order)
        orderAmount = self.getOrderAmount(order)
        quoteAmount = orderAmount * orderPrice
        if ( orderType == realTimePaperEngine.buyString):
            if ( orderPrice >= self.low):
                self.onBuyOrderDone(orderPrice, orderAmount, quoteAmount)
                return True
        if ( orderType == realTimePaperEngine.sellString):
            if (orderPrice <= self.high):
                self.onSellOrderDone(orderPrice, orderAmount, quoteAmount)
                return True
        return False
    def getOrderPrice(self, order):
        return order[2]
    def getOrderAmount(self, order):
        return order[1]
    def getOrderType(self, order):
        return order[3]
    def onCancelOrder(self, order):
        orderType = self.getOrderType(order)
        orderPrice = self.getOrderPrice(order)
        orderAmount = self.getOrderAmount(order)
        log("canceling {} order for {:.8f} at price {:.8f} ".format( orderType, orderAmount, orderPrice))

        if ( orderType == realTimePaperEngine.buyString):
            self.activeBuyOrder -= 1
        if ( orderType == realTimePaperEngine.sellString):
            self.activeSellOrder -= 1
            self.availableAsset += orderAmount
    def cancelOrder(self, order):
        self.onCancelOrder(order)
    def shouldSleep(self):
        return True

In [123]:
class realTimeEngine(realTimePaperEngine):
    def purgeCcxtCache(self):
        # keep last day of history in cache
        before = exchange.milliseconds () - 24 * 60 * 60 * 1000
        # purge all closed and canceled orders "older" or issued "before" that time
        exchange.purge_cached_orders (before)
    def getPrice(self):
        return (self.pairInfo['ask'] + self.pairInfo['bid'])/2.0
    def update(self, pair):
        #self.purgeCcxtCache()
        self.pairInfo = exchange.fetchTicker( pair )
        log("current close : {:.8f} - nb active Buy : {} - nb active Sell {} - nb completed Buy : {} - nb completed Sell {}"
                .format(self.getPrice(), self.activeBuyOrder, self.activeSellOrder, self.completedBuyOrder, self.completedSellOrder))
        return True
    def buy( self, pair, amount, price):
        log ( "putting buy order for {:.8f} at price {:.8f} ".format( amount, price), "green")
        order = exchange.create_limit_buy_order(pair, amount, price)
        self.activeBuyOrder += 1
        return [order, price, amount, realTimePaperEngine.buyString]
    def sell( self, pair, amount, price):
        log( "putting sell order for {:.8f} at price {:.8f} ".format( amount, price), "red")
        order = exchange.create_limit_sell_order(pair, amount, price)
        self.activeSellOrder += 1
        self.availableAsset -= amount
        return [order, price, amount, realTimePaperEngine.sellString]
    def isOrderDone( self, order):
        orderType = self.getOrderType(order)
        orderPrice = self.getOrderPrice(order)
        orderAmount = self.getOrderAmount(order)
        quoteAmount = orderAmount * orderPrice
        
        exchange_order = order[0]
        orderStatus = exchange.fetchOrder( exchange_order["id"], exchange_order["symbol"])
        
        if orderStatus["status"] == "closed":
            if ( orderType == realTimePaperEngine.buyString):
                self.onBuyOrderDone(orderPrice, orderAmount, quoteAmount)
                return True
            if ( orderType == realTimePaperEngine.sellString):
                self.onSellOrderDone(orderPrice, orderAmount, quoteAmount)
                return True
        return False
    def getOrderPrice(self, order):
        return order[1]
    def getOrderAmount(self, order):
        return order[2]
    def getOrderType(self, order):
        return order[3]
    def cancelOrder(self, order):
        exchange_order = order[0]
        exchange.cancelOrder( exchange_order["id"], exchange_order["symbol"])
        self.onCancelOrder(order)

    def shouldSleep(self):
        return True
        

In [124]:
class backtestEngine(realTimePaperEngine):
    def reset(self):
        self.candleId = 0
        super().reset()
    def uploadCandles( self, symbol, since ):
        all_candles = []
        while since < exchange.milliseconds ():
            limit = 500
            print(since)
            time.sleep(1)
            candles = exchange.fetch_ohlcv(symbol, "1m", since=since, limit=limit)
            print (len(candles))
            if len(candles):
                since = candles[len(candles) - 1][0] + 1
                all_candles += candles
            else:
                break
        self.candles = all_candles 
    def saveCandles( self, filename ):
        with open( filename, 'wb') as f:
            pickle.dump(self.candles, f)
    def loadCandles( self, filename):
        self.reset()
        with open( filename, 'rb') as f:
            self.candles = pickle.load(f)
    def update(self, pair):
        if self.candleId >= len ( self.candles) :
            log("***** End of simulation ! ****")
            self.walletDisplay()
            return False
        self.candle = self.candles[self.candleId]
        self.candleId += 1
        self.open  = self.candle[1]
        self.high  = self.candle[2]
        self.low   = self.candle[3]
        self.close = self.candle[4]
        log("current close : {:.8f} - nb active Buy : {} - nb active Sell {} - nb completed Buy : {} - nb completed Sell {}"
                .format(self.close, self.activeBuyOrder, self.activeSellOrder, self.completedBuyOrder, self.completedSellOrder))
        return True
    def shouldSleep(self):
        return False


In [125]:
class MarketMaker:
    def __init__( self, pair, spreadPercent, limit, engine):
        self.halfSpread = spreadPercent / 2.0
        self.buyOrderList= []
        self.sellOrderList = []
        self.openBuySellBalance = 0
        self.closedBuySellBalance = 0
        self.openSellLimit = limit
        self.amountQuote = 0.022
        self.pair = pair
        self.engine = engine
        
    def getAmount(self, price ):
        amount = self.amountQuote / price
        return amount
    def tryToBuy(self, price):
        amount = self.getAmount( price )
        buyPrice = price * ( 1.0 - self.halfSpread) 
        order = self.engine.buy( self.pair, amount, buyPrice)
        if ( order != None ):
            self.buyOrderList.append( order )
    def tryToSell(self, price, amount):
        #amount = self.getAmount( price )
        sellPrice = price * ( 1.0 + self.halfSpread )
        order = self.engine.sell( self.pair, amount, sellPrice)
        if ( order != None ):
            self.sellOrderList.append( order )
    def processBuyOrders(self):
        buyOrderDone = []
        for order in self.buyOrderList:
            if (self.engine.isOrderDone(order) ):
                buyOrderDone.append( order )
                self.closedBuySellBalance += 1
        for order in buyOrderDone:
            self.buyOrderList.remove(order)
        return buyOrderDone
    def processSellOrders(self):
        SellOrderDone = []
        for order in self.sellOrderList:
            if (self.engine.isOrderDone(order) ):
                SellOrderDone.append(order)
                self.closedBuySellBalance -= 1
        for order in SellOrderDone:
            self.sellOrderList.remove(order)
        return len(SellOrderDone) != 0
    
    def handleBuyList(self, currentPrice):
        if len(self.buyOrderList) == 0:
            self.tryToBuy(currentPrice)
            return
        if len ( self.buyOrderList) > 1:
            log("More than one item in buy list !!! {} items".format(len(self.buyOrderList)), "bgRed")
        self.updateBuyOrderPrice(currentPrice)
    def updateBuyOrderPrice(self, currentPrice):
        # put a higher price if the current price is too high :
        orderPrice = self.engine.getOrderPrice( self.buyOrderList[0] )
        limitPrice = currentPrice * ( 1 - 2.0 * self.halfSpread)
        log("AssetPrice:{:.8f}-OrderPrice : {:.8f}-limitPrice:{:.8f}".format(currentPrice, orderPrice, limitPrice))
        if ( orderPrice < limitPrice):
            self.engine.cancelOrder( self.buyOrderList[0] )
            self.buyOrderList.remove( self.buyOrderList[0] )
            self.tryToBuy(currentPrice)           

    def handleSellList(self, currentPrice):
        #sort list by dec price order :
        self.sellOrderList.sort( key = lambda o:self.engine.getOrderPrice(o), reverse=True)
        self.engine.cancelOrder( self.sellOrderList[0] )
        self.sellOrderList.remove( self.sellOrderList[0] )
    
    def addOrders(self, currentPrice):
        amount = self.getAmount(currentPrice)
        # sell if possible :
        if ( amount < self.engine.getAvailableAsset() ):
            self.tryToSell( currentPrice, amount)
        # buy ( todo : if possible ):
        if len(self.buyOrderList) > 0:
            self.engine.cancelOrder( self.buyOrderList[0] )
            self.buyOrderList.remove( self.buyOrderList[0] )
        self.tryToBuy(currentPrice)           
        

    def update(self):
        if self.engine.update(self.pair) == False:
            return False
        
        currentPrice = self.engine.getPrice()
        SellDone = self.processSellOrders()
        BuyDone = self.processBuyOrders()

        if SellDone:
            # if both sell and buy occured, the buy list can be empty :
            if len(self.buyOrderList) > 0:
                self.engine.cancelOrder( self.buyOrderList[0] )
                self.buyOrderList.remove( self.buyOrderList[0] )
            self.tryToBuy(currentPrice)
            amount = self.getAmount(currentPrice)
            # sell if possible :
            if ( amount < self.engine.getAvailableAsset() ):
                self.tryToSell( currentPrice, amount)
        if len(BuyDone) > 0:
            self.tryToSell( self.engine.getOrderPrice(BuyDone[0]), self.engine.getOrderAmount(BuyDone[0]) )
            if not SellDone:
                self.tryToBuy( currentPrice )
#        if SellDone or len(BuyDone) > 0:
#            self.addOrders(currentPrice)

        #if len(self.sellOrderList) == 0:
        self.handleBuyList( currentPrice )
        if  len( self.sellOrderList ) >= self.openSellLimit:
            self.handleSellList( currentPrice )
        return True
        
    def run(self):
        log("Starting the bot...")
        while( True ):
            try:
                result = self.update()
                if result == False:
                    break
            # sleep / interrupt management :
            except KeyboardInterrupt:
                assert False
            except Exception as e:
                print ("Exception : " + str(e) + str(traceback.format_exc()) )
                time.sleep(10)
            if self.engine.shouldSleep():
                for i in range(60):    # Sleep divided to have quicker keyboard interruptions
                    time.sleep(1)